In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [52]:
df_lstm=pd.read_csv('apple_data.csv')
df_lstm.head()

,Date,Close,High,Low,Open,Volume
0,2015-01-02,24.288580,24.757334,23.848706,24.746226,212818400
1,2015-01-05,23.604326,24.137507,23.417714,24.057530,257142000
2,2015-01-06,23.606552,23.866477,23.244433,23.668756,263188400
3,2015-01-07,23.937576,24.037547,23.704309,23.815388,160423600
4,2015-01-08,24.857306,24.915067,24.148620,24.266365,237458000


In [ ]:
df_lstm.shape

(2515, 6)

In [54]:
df_lstm['Date'] = pd.to_datetime(df_lstm['Date'])
df_lstm.set_index('Date', inplace=True)
df_lstm = df_lstm.asfreq('B')

Handling Missing Values

In [55]:
df_lstm['Close'] = df_lstm['Close'].interpolate(method='linear')


Lag Features

In [ ]:
for lag in range(1, 4):
    df_lstm[f'lag_{lag}'] = df_lstm['Close'].shift(lag)


In [57]:
df_lstm.head()

,Close,High,Low,Open,Volume,lag_1,lag_2,lag_3
Date,,,,,,,,
2015-01-02,24.288580,24.757334,23.848706,24.746226,212818400.0,NaN,NaN,NaN
2015-01-05,23.604326,24.137507,23.417714,24.057530,257142000.0,24.288580,NaN,NaN
2015-01-06,23.606552,23.866477,23.244433,23.668756,263188400.0,23.604326,24.288580,NaN
2015-01-07,23.937576,24.037547,23.704309,23.815388,160423600.0,23.606552,23.604326,24.288580
2015-01-08,24.857306,24.915067,24.148620,24.266365,237458000.0,23.937576,23.606552,23.604326


Rolling Statistics

In [58]:
df_lstm['rolling_mean_7'] = df_lstm['Close'].rolling(window=7).mean()
df_lstm['rolling_std_7']  = df_lstm['Close'].rolling(window=7).std()


In [59]:
df_lstm.head()

,Close,High,Low,Open,Volume,lag_1,lag_2,lag_3,rolling_mean_7,rolling_std_7
Date,,,,,,,,,,
2015-01-02,24.288580,24.757334,23.848706,24.746226,212818400.0,NaN,NaN,NaN,NaN,NaN
2015-01-05,23.604326,24.137507,23.417714,24.057530,257142000.0,24.288580,NaN,NaN,NaN,NaN
2015-01-06,23.606552,23.866477,23.244433,23.668756,263188400.0,23.604326,24.288580,NaN,NaN,NaN
2015-01-07,23.937576,24.037547,23.704309,23.815388,160423600.0,23.606552,23.604326,24.288580,NaN,NaN
2015-01-08,24.857306,24.915067,24.148620,24.266365,237458000.0,23.937576,23.606552,23.604326,NaN,NaN


In [60]:
df_lstm.isnull().sum()

Close              0
High              92
Low               92
Open              92
Volume            92
lag_1              1
lag_2              2
lag_3              3
rolling_mean_7     6
rolling_std_7      6
dtype: int64

Target Variable

In [61]:
df_lstm['target'] = df_lstm['Close'].shift(-1)

In [62]:
df_lstm.head()

,Close,High,Low,Open,Volume,lag_1,lag_2,lag_3,rolling_mean_7,rolling_std_7,target
Date,,,,,,,,,,,
2015-01-02,24.288580,24.757334,23.848706,24.746226,212818400.0,NaN,NaN,NaN,NaN,NaN,23.604326
2015-01-05,23.604326,24.137507,23.417714,24.057530,257142000.0,24.288580,NaN,NaN,NaN,NaN,23.606552
2015-01-06,23.606552,23.866477,23.244433,23.668756,263188400.0,23.604326,24.288580,NaN,NaN,NaN,23.937576
2015-01-07,23.937576,24.037547,23.704309,23.815388,160423600.0,23.606552,23.604326,24.288580,NaN,NaN,24.857306
2015-01-08,24.857306,24.915067,24.148620,24.266365,237458000.0,23.937576,23.606552,23.604326,NaN,NaN,24.883959


Dropping NaN Rows

In [63]:
df_lstm_final = df_lstm[['Close', 'lag_1', 'lag_2', 'lag_3', 'rolling_mean_7', 'rolling_std_7', 'target']].dropna()

In [64]:
df_lstm_final.isnull().sum()

Close             0
lag_1             0
lag_2             0
lag_3             0
rolling_mean_7    0
rolling_std_7     0
target            0
dtype: int64

In [68]:
df_lstm_final.describe()

,Close,lag_1,lag_2,lag_3,rolling_mean_7,rolling_std_7,target
count,2600.000000,2600.000000,2600.000000,2600.000000,2600.000000,2600.000000,2600.000000
mean,0.309103,0.308731,0.308884,0.309037,0.310861,0.163758,94.223827
std,0.275441,0.275180,0.275379,0.275580,0.278722,0.155284,65.544296
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.647446
25%,0.062668,0.062589,0.062632,0.062591,0.062083,0.041983,35.571258
50%,0.185869,0.185394,0.185413,0.185506,0.183924,0.111126,64.932724
75%,0.546588,0.546386,0.547265,0.547060,0.551083,0.242644,150.630825
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,258.396667


In [65]:
df_lstm_final.shape

(2600, 7)

Feature Scaling

In [66]:
scaler = MinMaxScaler()
features = ['Close', 'lag_1', 'lag_2', 'lag_3', 'rolling_mean_7', 'rolling_std_7']
df_lstm_final[features] = scaler.fit_transform(df_lstm_final[features])


In [ ]:
df_lstm_final.describe()

,Close,lag_1,lag_2,lag_3,rolling_mean_7,rolling_std_7,target
count,2600.000000,2600.000000,2600.000000,2600.000000,2600.000000,2600.000000,2600.000000
mean,0.309103,0.308731,0.308884,0.309037,0.310861,0.163758,94.223827
std,0.275441,0.275180,0.275379,0.275580,0.278722,0.155284,65.544296
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.647446
25%,0.062668,0.062589,0.062632,0.062591,0.062083,0.041983,35.571258
50%,0.185869,0.185394,0.185413,0.185506,0.183924,0.111126,64.932724
75%,0.546588,0.546386,0.547265,0.547060,0.551083,0.242644,150.630825
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,258.396667
